[View in Colaboratory](https://colab.research.google.com/github/yala/MLCodeLab/blob/master/lab2/cifar_exercise.ipynb)

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

0.4.0
False


# The Task: CIFAR Classification
<img src="https://raw.githubusercontent.com/yala/MLCodeLab/master/lab2/cifar.png">

In this lab, we'll build a neural network to classify images on CIFAR 10.



## Step 1: Loading Data and Preprocessing
Let's start by loading the data.
We're going to normalize our images to have 0 mean, and unit variance. We'll do this using some torchvision transforms. This generally helps stablize learning, and is common practice. 

In [0]:
normalize_image = transforms.Compose([
                           transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))   
                       ])

all_train = datasets.CIFAR10('data', train=True, download=True,
                        transform=normalize_image)
num_train = int(len(all_train)*.8)
train = [all_train[i] for i in range(num_train)]
dev = [all_train[i] for i in range(num_train,len(all_train))]
test = datasets.CIFAR10('data', train=False, download=True, 
                      transform=normalize_image)
                           


Files already downloaded and verified
Files already downloaded and verified


In [0]:
train[0][0].size()

torch.Size([3, 32, 32])

## Step 2: Building a model

All pytorch models should be implemented as instances of `nn.Module`. 

To build a model you need to:
a) define what parameters it'll need in it's `__init__` function
b) define the model's computation, using those parameters, in a forward function.


Experiment with different neural architectures. I recommend starting with a simple linear model and building up from there. Here, I implement a simple 4-layer convolutional neural network. 

In [0]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        ## TODO Define your model
        
    def forward(self, x):
        # TODO Define the forward function of your model
        return x
    

## Step 3. Defining our training procedure

Set your batch size, learning rate, number of epochs etc. Experiment with various hyper parameters.


In [0]:
# Training settings
batch_size =  # TODO, experiment with different values
epochs = # TODO, experiment with different values
lr = # TODO, experiment with different values
momentum = # TODO, experiment with different values


train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)


model = Model()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum) # TODO, Try Adam Optimizer or other optimizers.

In [0]:

def train_epoch( model, train_loader, optimizer, epoch):
    model.train() # Set the nn.Module to train mode. 
    total_loss = 0
    correct = 0
    num_samples = len(train_loader.dataset)
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        total_loss += loss.detach() # Don't keep computation graph 

    print('Train Epoch: {} \tLoss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            epoch, total_loss / num_samples, 
            correct, 
            num_samples,
            100. * correct / num_samples))

def eval_epoch(model, test_loader, name):
    model.eval()
    test_loss = 0
    correct = 0
    for x, target in test_loader:
        output = model(x)
        test_loss += F.cross_entropy(output, target).item() # sum up batch loss
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name,
        test_loss, 
        correct, 
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


## Step 4: Training the model

In [0]:

for epoch in range(1, epochs + 1):
    train_epoch(model, train_loader, optimizer, epoch)
    eval_epoch(model,  dev_loader, "Dev")
    print("---")


## Run your best model on Test

In [0]:
eval_epoch(model,  test_loader, "Test")